In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
print(tf.__version__)

2.0.0-rc1


In [3]:
np.random.seed(7)
top_words = 10000
max_review_length = 80
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=top_words)

In [7]:
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_review_length)
x_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_review_length)

print(x_train.shape)

(25000, 80)


In [8]:
class RNN(keras.Model):
    def __init__(self, units, num_classes, num_layers):
        super(RNN, self).__init__()
        # self.cells = [keras.layers.LSTMCell(units) for _ in range(num_layers)]
        # self.rnn = keras.layers.RNN(self.cells, unroll=True)
        self.rnn = keras.layers.LSTM(units, return_sequences=True)
        self.rnn2 = keras.layers.LSTM(units)

        # self.cells = (keras.layers.LSTMCell(units) for _ in range(num_layers))
        # self.rnn = keras.layers.RNN(self.cells, return_sequences=True, return_state=True)
        # self.rnn = keras.layers.LSTM(units, unroll=True)
        # self.rnn = keras.layers.StackedRNNCells(self.cells)

        # have 1000 words totally, every word will be embedding into 100 length vector
        # the max sentence lenght is 80 words
        self.embedding = keras.layers.Embedding(top_words, 100, input_length=max_review_length)
        self.fc = keras.layers.Dense(1)

    def call(self, inputs, training=None, mask=None):
        # print('x', inputs.shape)
        # [b, sentence len] => [b, sentence len, word embedding]
        x = self.embedding(inputs)
        # print('embedding', x.shape)
        x = self.rnn(x) 
        x = self.rnn2(x) 
        # print('rnn', x.shape)
        x = self.fc(x)
        print(x.shape)

        return x

In [9]:
units = 64
num_classes = 2
batch_size = 32
epochs = 5

model = RNN(units, num_classes, num_layers=2)
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_data=(x_test, y_test), verbose=1)

# evaluate on test set
acc = model.evaluate(x_test, y_test, batch_size, verbose=1)
print("Final test loss and accuracy :", acc)

(None, 1)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
(None, 1)
(None, 1)
25000/25000 [==============================] - 58s 2ms/sample - loss: 0.4282 - accuracy: 0.7851 - val_loss: 0.3832 - val_accuracy: 0.8208
Epoch 2/5
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.2707 - accuracy: 0.8845 - val_loss: 0.3767 - val_accuracy: 0.8106
Epoch 3/5
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.1926 - accuracy: 0.9223 - val_loss: 0.3998 - val_accuracy: 0.7983
Epoch 4/5
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.1282 - accuracy: 0.9516 - val_loss: 0.5456 - val_accuracy: 0.8264
Epoch 5/5
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.0871 - accuracy: 0.9692 - val_loss: 0.5958 - val_accuracy: 0.8305
25000/1 [=============================================================================

Final test loss and accuracy : [0.5958489277517796, 0.83048]
